# Detection of other arm activities
This notebook shows how to predict the probability of other arm activities during gait using extracted features from windowed accelerometer and gyroscope data as input.

## Modules

In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import pandas as pd
import tsdf

from dbpd import DataColumns

## Constants

In [ ]:
# Cell has the tag 'parameters'

path_to_data = '../../../tests/data'
input_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
path_output = os.path.join(path_to_data, '4.predictions', 'gait')

In [2]:
# paths and files
path_to_classifier_input = os.path.join(path_to_data, '0.classifiers', 'gait')

classifier_file_name = 'asd_classifier.pkl'

meta_filename = 'arm_swing_meta.json'
time_filename = 'arm_swing_time.bin'
values_filename = 'arm_swing_values.bin'

l_accel_cols = [DataColumns.ACCELERATION_X, DataColumns.ACCELERATION_Y, DataColumns.ACCELERATION_Z]
l_gyro_cols = [DataColumns.ROTATION_X, DataColumns.ROTATION_Y, DataColumns.ROTATION_Z]

AttributeError: type object 'DataColumns' has no attribute 'ACCELERATION_X'

## Load data

#### Features

In [ ]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(input_path, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.head(2)

,time,angle_perc_power,range_of_motion,forward_peak_ang_vel_mean,forward_peak_ang_vel_std,backward_peak_ang_vel_mean,backward_peak_ang_vel_std,std_norm_acc,grav_accelerometer_x_mean,grav_accelerometer_x_std,...,cc_7_gyroscope,cc_8_gyroscope,cc_9_gyroscope,cc_10_gyroscope,cc_11_gyroscope,cc_12_gyroscope,cc_13_gyroscope,cc_14_gyroscope,cc_15_gyroscope,cc_16_gyroscope
0,0.00,0.933953,2.875672,8.193249,3.292503,4.938885,0.0,0.154163,-0.254277,0.201148,...,0.911306,3.910365,2.699518,2.580446,1.602991,0.556265,-1.046647,0.541998,-0.346748,0.365782
1,0.75,0.992147,4.295439,3.288568,1.612178,4.401343,0.0,0.160148,-0.395660,0.176619,...,2.608440,4.632687,2.083540,2.834911,1.474186,-0.073377,-2.122460,0.373824,1.433426,1.711163


#### Classifier

In [ ]:
clf = pd.read_pickle(os.path.join(path_to_classifier_input, classifier_file_name))

## Prepare data

In [ ]:
clf.feature_names_in_ = [f'{x}_power_below_gait' for x in l_accel_cols] + \
                        [f'{x}_power_gait' for x in l_accel_cols] + \
                        [f'{x}_power_tremor' for x in l_accel_cols] + \
                        [f'{x}_power_above_tremor' for x in l_accel_cols] + \
                        [f'grav_{x}_{y}' for x in l_accel_cols for y in ['mean', 'std']] + \
                        ['std_norm_acc'] + [f'cc_{i}_acceleration' for i in range(1, 17)] + \
                        [f'{x}_dominant_frequency' for x in l_accel_cols] + \
                        [f'cc_{i}_rotation' for i in range(1, 17)] + \
                        [f'{x}_dominant_frequency' for x in l_gyro_cols] + \
                        ['range_of_motion', 'forward_peak_ang_vel_mean',
                         'forward_peak_ang_vel_std', 'backward_peak_ang_vel_mean',
                         'backward_peak_ang_vel_std', 'angle_perc_power']

X = df.loc[:, clf.feature_names_in_]

KeyError: "['acceleration_x_power_below_gait', 'acceleration_y_power_below_gait', 'acceleration_z_power_below_gait', 'acceleration_x_power_gait', 'acceleration_y_power_gait', 'acceleration_z_power_gait', 'acceleration_x_power_tremor', 'acceleration_y_power_tremor', 'acceleration_z_power_tremor', 'acceleration_x_power_above_tremor', 'acceleration_y_power_above_tremor', 'acceleration_z_power_above_tremor', 'grav_acceleration_x_mean', 'grav_acceleration_x_std', 'grav_acceleration_y_mean', 'grav_acceleration_y_std', 'grav_acceleration_z_mean', 'grav_acceleration_z_std', 'cc_1_acceleration', 'cc_2_acceleration', 'cc_3_acceleration', 'cc_4_acceleration', 'cc_5_acceleration', 'cc_6_acceleration', 'cc_7_acceleration', 'cc_8_acceleration', 'cc_9_acceleration', 'cc_10_acceleration', 'cc_11_acceleration', 'cc_12_acceleration', 'cc_13_acceleration', 'cc_14_acceleration', 'cc_15_acceleration', 'cc_16_acceleration', 'acceleration_x_dominant_frequency', 'acceleration_y_dominant_frequency', 'accelerat

## Predict 

In [ ]:
# df['pred_arm_swing_proba'] = clf.predict_proba(X)[:, 1]
df['pred_arm_swing'] = clf.predict(X)

## Store predictions

In [ ]:
metadata_samples.__setattr__('file_name', 'arm_swing_values.bin')
metadata_samples.__setattr__('file_dir_path', path_output)
metadata_time.__setattr__('file_name', 'arm_swing_time.bin')
metadata_time.__setattr__('file_dir_path', path_output)

metadata_samples.__setattr__('channels', ['pred_arm_swing'])
metadata_samples.__setattr__('units', ['boolean'])
metadata_samples.__setattr__('data_type', np.int8)
metadata_samples.__setattr__('bits', 8)

metadata_time.__setattr__('channels', ['time'])
metadata_time.__setattr__('units', ['relative_time_ms'])
metadata_time.__setattr__('data_type', np.int32)
metadata_time.__setattr__('bits', 32)

In [ ]:
if not os.path.exists(path_output):
    os.makedirs(path_output)

# store binaries and metadata
tsdf.write_dataframe_to_binaries(path_output, df, [metadata_time, metadata_samples])
tsdf.write_metadata([metadata_time, metadata_samples], 'arm_swing_meta.json')